<a href="https://colab.research.google.com/github/deutranium/tfx-addons/blob/main/tfx_addons/feature_selection/example/Iris_example_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFX Feature Selection Component

You may find the source code for the same [here](https://github.com/tensorflow/tfx-addons/tree/main/tfx_addons/feature_selection)

This example demonstrate the use of feature selection component. This project allows the user to select different algorithms for performing feature selection on datasets artifacts in TFX pipelines

Base code taken from: https://github.com/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb

## Setup
### Install TFX
_Note: In Google Colab, because of package updates, the first time you run this cell you must restart the runtime (Runtime > Restart runtime ...)._

In [1]:
!pip install -U tfx

/home/x000ff4/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/x000ff4/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/home/x000ff4/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
# getting the code directly from the repo
x = !pwd

if 'feature_selection' not in str(x):
  !git clone  -b main https://github.com/deutranium/tfx-addons.git
  %cd tfx-addons/tfx_addons/feature_selection

fatal: destination path 'tfx-addons' already exists and is not an empty directory.
/home/x000ff4/open_source/tfx-addons/tfx_addons/example_filter/tfx-addons/tfx_addons/feature_selection


### Import packages
Importing the necessary packages, including the standard TFX component classes

In [3]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False


pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

# importing the feature selection component
from component import MyTrainerComponent


# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

## Palmer Penguins example pipeline

### Download Example Data

We download the example dataset for use in our TFX pipeline.

The dataset we're using is the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) which is also used in other
[TFX examples](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

There are four numeric features in this dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

All features were already normalized to have range [0,1]. We will build a
that selects 2 features to be eliminated from the dataset in other to improve the performance of the mode in predicting the `species` of penguins.

In [4]:
# getting the dataset
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
             
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datar6p9_9qf/data.csv', <http.client.HTTPMessage at 0x7fcc35c4aca0>)

### Run TFX Components
In the cells that follow, we create TFX components one-by-one and generates `example` using `exampleGen` component.

In [5]:
context = InteractiveContext()

#create and run exampleGen component
example_gen = CsvExampleGen(input_base=_data_root )
context.run(example_gen)

#create and run statisticsGen component
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [6]:
# using the feature selection component
#feature selection component


feature_selector = MyTrainerComponent(training_data = example_gen.outputs['examples'],filter_function_str='filter_function')#.uri+"/Split-train"
context.run(feature_selector)

-------
['PROPERTIES', 'TYPE_ANNOTATION', 'TYPE_NAME', '_MLMD_ARTIFACT_TYPE', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_artifact', '_artifact_type', '_cached_json_value_custom_properties', '_cached_json_value_properties', '_get_artifact_type', '_get_system_property', '_initialized', '_set_system_property', 'artifact_type', 'copy_from', 'from_json_dict', 'get_custom_property', 'get_float_custom_property', 'get_int_custom_property', 'get_json_value_custom_property', 'get_string_custom_property', 'has_custom_property', 'id', 'mlmd_artifact', 'name', 'pipeline_name', 'producer_component', 'set

2022-10-12 18:36:10.639574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 18:36:10.644854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 18:36:10.644988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 18:36:10.645534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

ExecutionResult(
    component_id: MyTrainerComponent
    execution_id: 3
    outputs:
        filtered_data: OutputChannel(artifact_type=Examples, producer_component_id=MyTrainerComponent, output_key=filtered_data, additional_properties={}, additional_custom_properties={})
        list_len: OutputChannel(artifact_type=Integer, producer_component_id=MyTrainerComponent, output_key=list_len, additional_properties={}, additional_custom_properties={}))

In [7]:
# Display Selected Features
context.show(feature_selector.outputs['filtered_data']._artifacts[0])

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,/tmp/tfx-interactive-2022-10-12T18_36_07.202387-6d6c6dc9/MyTrainerComponent/filtered_data/3
.span,0
.split_names,
.version,0


As seen above, `.selected_features` contains the features selected after running the component with the speified parameters.

To get the info about updated `Example` artifact, one can view it as follows:

In [8]:
context.show(feature_selector.outputs['updated_data']._artifacts[0])

KeyError: 'updated_data'